In [2]:
!pip install requests bs4 PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


In [4]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 977.0 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [4]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
import openai
import re
import time

openai.api_key ="your_api_key"

pdf_path = 'Blog.pdf'
pdf_text = ''
with open(pdf_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        pdf_text += page.extract_text()

url='https://blog.google/products/gemini/google-gemini-new-features-july-2024/'
txt = requests.get(url)
soup = BeautifulSoup(txt.text, 'html.parser')
webpage_text = ' '.join([p.text for p in soup.find_all('p')])
words = webpage_text.split()
first_4000_words = ' '.join(words[:4000])


def call_openai_api(messages):
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=messages
            )
            return response
        except openai.error.RateLimitError:
            print("Rate limit reached. Retrying in 5 seconds...")
            time.sleep(10)

# Define the chatbot function
def chatbot_conversation():
    messages = []

    messages.append({"role": "user", "content": "Solve this complex math problem: Evaluate the integral ∫(x^3 + 2x^2 - x + 1) dx."})
    response = call_openai_api(messages)
    print("Step 1: Complex Math Problem")
    print(response.choices[0].message['content'])
    messages.append({"role": "assistant", "content": response.choices[0].message['content']})

    pdf_prompt = f'You are given a PDF content {pdf_text}. Rewrite the summary and answer the following question shortly. Q1: How can data scientists effectively balance the benefits of GenAI with the potential risks? Q2:What specific strategies can data science teams implement to leverage AI effectively within their organizations?'
    messages.append({"role": "user", "content": pdf_prompt})
    response = call_openai_api(messages)
    print("\nStep 2: PDF Analysis")
    print(response.choices[0].message['content'])
    messages.append({"role": "assistant", "content": response.choices[0].message['content']})

    web_prompt = f'You are given a website URL content {first_4000_words}. Rewrite the summary from URL and answer the following question shortly. Q1: What are the potential risks of using Google Gemini? Q2: What are the benefits of using Google Gemini?'
    messages.append({"role": "user", "content": web_prompt})
    response = call_openai_api(messages)
    print("\nStep 3: Website Analysis")
    print(response.choices[0].message['content'])
    messages.append({"role": "assistant", "content": response.choices[0].message['content']})

    messages.append({"role": "user", "content": "Please summarize our entire conversation."})
    response = call_openai_api(messages)
    print("\nStep 3: Chat Summary")
    print(response.choices[0].message['content'])

chatbot_conversation()


Step 1: Complex Math Problem
The integral of a polynomial is found by raising the power of each term by one and dividing by the new power. Therefore, the integral of the given polynomial is:

∫(x^3 + 2x^2 - x + 1) dx = (1/4)x^4 + (2/3)x^3 - (1/2)x^2 + x + C,

where C is the constant of integration.

Step 2: PDF Analysis
Summary: The use of Generative AI (GenAI) is transforming data science teams, enabling faster development cycles, less technical and maintenance efforts, and fostering unique application cases. Nonetheless, GenAI also introduces new risks. The shift requires data scientists to approach their roles differently. It has resulted in the appearance of AI projects that are implemented by external model providers and integrated by non-AI expert teams. However, data scientists remain crucial for verifying when AI usage is appropriate and creating reliable solutions. AI literacy is becoming significant within organizations and data scientists play a pivotal role in making this a